# Import libraries

In [ ]:
import pandas as pd 
import numpy as np 
from collections import deque 
import plotly.graph_objs as go
import plotly.offline as py


# Load Data :

In [3]:
# Load TMDB data
df1=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
df1.columns = ['id','Name','cast','crew']
df2= df2.merge(df1,on='id')
print(df1.shape)
print(df2.shape)
print(df2.head(3))

(4803, 4)
(4803, 23)
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                  

In [4]:
# Load Netflix data for movie titles :
movie_titles = pd.read_csv('../input/netflix-prize-data/movie_titles.csv', 
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['Id', 'Year', 'Name']).set_index('Id')

print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))
print(movie_titles.head(5))

Shape Movie-Titles:	(17770, 2)
      Year                          Name
Id                                      
1   2003.0               Dinosaur Planet
2   2004.0    Isle of Man TT 2004 Review
3   1997.0                     Character
4   1994.0  Paula Abdul's Get Up & Dance
5   2004.0      The Rise and Fall of ECW


In [5]:
# Load Netflix data for user data (movie ID is indexed):
df_raw1 = pd.read_csv('../input/netflix-prize-data/combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])
#df_raw2 = pd.read_csv('../input/netflix-prize-data/combined_data_2.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])
#df_raw3 = pd.read_csv('../input/netflix-prize-data/combined_data_3.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])
#df_raw4 = pd.read_csv('../input/netflix-prize-data/combined_data_4.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])

df_raw = df_raw1
#df_raw = df_raw.append(df_raw2)
#df_raw = df_raw.append(df_raw3)
#df_raw = df_raw.append(df_raw4)

df_raw.index = np.arange(0,len(df_raw))
#print(df_raw.index)    #RangeIndex(start=0, stop=24058263, step=1)

df_raw['Rating'] = df_raw['Rating'].astype(float)

print('Dataset 1 shape: {}'.format(df_raw.shape))
print('-Dataset examples-')
print(df_raw.iloc[::5000000, :])

Dataset 1 shape: (24058263, 3)
-Dataset examples-
             User  Rating        Date
0              1:     NaN         NaN
5000000   2560324     4.0  2005-12-06
10000000  2271935     2.0  2005-04-11
15000000  1921803     2.0  2005-01-31
20000000  1933327     3.0  2004-11-10


In [6]:
# Load Movies Dataset's movies metadata :
movie_metadata = pd.read_csv('../input/the-movies-dataset/movies_metadata.csv', low_memory=False)[['original_title', 'overview', 'vote_count']].set_index('original_title').dropna()
# Remove the long tail of rarly rated moves
movie_metadata = movie_metadata[movie_metadata['vote_count']>10].drop('vote_count', axis=1)

print('Shape Movie-Metadata:\t{}'.format(movie_metadata.shape))
movie_metadata.sample(5)

Shape Movie-Metadata:	(21604, 1)


,overview
original_title,
The Wild Geese,A British multinational company seeks to overt...
Odd Girl Out,A mother and her daughter confront the intimid...
Dracula III: Legacy,"In the near future, Uffizi and Luke travel to ..."
Chasing Papi,Playboy Thomas Fuentes has so far been able to...
The Glass Key,"During the campaign for reelection, the crooke..."


# Preprocessing

To add movie ID column to df.

In [7]:
# Find empty rows to slice dataframe for each movie
tmp_movies = df_raw[df_raw['Rating'].isna()]['User'].reset_index()
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values] #movie[:-1] everything except last item in array

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = df_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df = pd.concat(user_data)
del user_data, df_raw, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df.shape))
df.sample(5)

Shape User-Ratings:	(24053764, 4)


,User,Rating,Date,Movie
8706449,1107022,3.0,2004-02-04,1744
12470750,763484,1.0,2005-10-31,2391
20458080,636867,2.0,2003-03-21,3886
3596417,1209557,5.0,2005-09-11,692
10482624,1860870,5.0,2005-12-14,2033


In [47]:
#Extract vote count and vote average      df.rename(index={0: "x", 1: "y", 2: "z"})
tmp=pd.DataFrame(df.groupby('Movie').agg({'Rating':['count', 'mean']}))
#tmp=tmp.rename(index={0: "vote_count",1 :"vote_average"})
tmp.head()

Rating          
       count      mean
Movie                 
1        547  3.749543
2        145  3.558621
3       2012  3.641153
4        142  2.739437
5       1140  3.919298

In [9]:
#Adding vote count and vote averages
df_n=df.merge(tmp,on='Movie')  ####check m
#df_n.drop('Rating_y',axis='columns', inplace=True)
df_n.columns=['User', 'Rating', 'Date', 'Movie', 'vote_count', 'vote_average']
print(df_n.head())
print(df_n.shape)

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



      User  Rating        Date  Movie  vote_count  vote_average
0  1488844     3.0  2005-09-06      1         547      3.749543
1   822109     5.0  2005-05-13      1         547      3.749543
2   885013     4.0  2005-10-19      1         547      3.749543
3    30878     4.0  2005-12-26      1         547      3.749543
4   823519     3.0  2004-05-03      1         547      3.749543
(24053764, 6)


In [10]:
#Join Netflix movie titles with df_n
movie_titles.index.names = ['Movie']
df_n=df_n.merge(movie_titles , on='Movie')
print(df_n.sample(5))
print(df_n.shape)

             User  Rating        Date  Movie  vote_count  vote_average  \
21690115  1147641     1.0  2003-02-14   4085        1363      3.392517   
8403706    274313     3.0  2004-12-24   1672         346      3.488439   
9705079    282492     4.0  2005-11-22   1905      193941      4.153908   
20824041  2339008     4.0  2004-05-17   3925      106807      3.514704   
13691810  2356670     4.0  2005-09-02   2612       87139      3.406316   

            Year                                               Name  
21690115  1958.0                                  Horror of Dracula  
8403706   2002.0                                Mujhse Dosti Karoge  
9705079   2003.0  Pirates of the Caribbean: The Curse of the Bla...  
20824041  2003.0                               The Matrix: Reloaded  
13691810  2001.0                                   Don't Say a Word  
(24053764, 8)


In [11]:
#Join with movies metadata dataset
movie_metadata.reset_index(inplace=True)
movie_metadata.rename(columns={"original_title": "Name"},inplace=True)

df_n=pd.merge(df_n, movie_metadata, on='Name')
#df_n.merge(movie_metadata, on=['Name', 'original_title'])
print(df_n.sample(5))
print(df_n.shape)

             User  Rating        Date  Movie  vote_count  vote_average  \
1327394    926860     2.0  2005-04-22    330       27038      3.460944   
4305528    564989     4.0  2003-09-30    985       89999      3.629029   
8826703   1365379     5.0  2005-08-20   1939       11634      3.448255   
17183029    79194     2.0  2004-04-27   3756       93241      3.059384   
13602451  2050892     4.0  2005-05-25   3030        5996      3.554536   

            Year                   Name  \
1327394   1998.0            Wild Things   
4305528   1999.0              The Mummy   
8826703   1979.0  The Amityville Horror   
17183029  2002.0          About Schmidt   
13602451  2001.0    The Mists of Avalon   

                                                   overview  
1327394   When teen-socialite Kelly Van Ryan (Richards) ...  
4305528   An ancient Egyptian priest called Imhotep is r...  
8826703   George Lutz and his wife Kathleen, move into t...  
17183029  66-year-old Warren Schmidt is a retire

In [12]:
print(df.shape)
df_n.shape

(24053764, 4)


(21141018, 9)

# Visualization

In [13]:
# Get data
data = movie_titles['Year'].value_counts().sort_index()

# Create trace
trace = go.Scatter(x = data.index,
                   y = data.values,
                   marker = dict(color = '#db0000'))
# Create layout
layout = dict(title = '{} Movies Grouped By Year Of Release'.format(movie_titles.shape[0]),
              xaxis = dict(title = 'Release Year'),
              yaxis = dict(title = 'Movies'))

# Create plot
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

In [14]:
# Get data
data = df['Rating'].value_counts().sort_index(ascending=False)

# Create trace
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               marker = dict(color = '#db0000'))
# Create layout
layout = dict(title = 'Distribution Of {} Netflix-Ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

In [15]:
# Get data
data = df['Date'].value_counts()
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)

# Create trace
trace = go.Scatter(x = data.index,
                   y = data.values,
                   marker = dict(color = '#db0000'))
# Create layout
layout = dict(title = '{} Movie-Ratings Grouped By Day'.format(df.shape[0]),
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Ratings'))

# Create plot
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

In [16]:
##### Ratings Per Movie #####
# Get data
data = df.groupby('Movie')['Rating'].count()#.clip(upper=99999)




#import seaborn as sns
#sns.set()#_theme(style="whitegrid")
#tips = sns.load_dataset("tips")
#ax = sns.scatterplot(x=data.values)



# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 1000000,
                                  size = 100),
                    marker = dict(color = '#db0000'))
# Create layout
layout = go.Layout(title = 'Distribution Of Ratings Per Movie #(Clipped at 99999)',
                   xaxis = dict(title = 'Ratings Per Movie'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
#fig = go.Figure(data=[trace], layout=layout)
#py.iplot(fig)



##### Ratings Per User #####
# Get data
data = df.groupby('User')['Rating'].count().clip(upper=1999)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 2000,
                                  size = 2),
                     marker = dict(color = '#db0000'))
# Create layout
layout = go.Layout(title = 'Distribution Of Ratings Per User (Clipped at 1999)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig)

# Weighted Rating

**Demographic filtering** for users that are new or we have less data on.


We can use the average ratings of the movie as the score but using this won't be fair enough since a movie with 8.9 average rating and only 3 votes cannot be considered better than the movie with 7.8 as as average rating but 40 votes. So, I'll be using IMDB's weighted rating (wr) which is given as :-

 ![](https://image.ibb.co/jYWZp9/wr.png)

where,
* v is the number of votes for the movie;
* m is the minimum votes required to be listed in the chart;
* R is the average rating of the movie; And
* C is the mean vote across the whole report

We already have v(**vote_count**) and R (**vote_average**) and C can be calculated as 

In [60]:
# Calculation based on the IMDB formula :
#weighted rating= (v/(v+m) * R) + (m/(m+v) * C)

#C=df['Rating'].mean()   #3.5996343025565563
#C

C=tmp.iloc[:,1].mean()
C

3.2219446775168796

In [39]:
#m=df.groupby('Movie')['Rating'].count()
#m=m.quantile(0.9)
#m
m=tmp.iloc[:, 0].quantile(0.9) 
m

11422.800000000017

In [68]:
# Calculation based on the IMDB formula
v = tmp.iloc[:, 0]
R = tmp.iloc[:, 1]
tmp['score']=((v/(v+m) * R) + (m/(m+v) * C))
tmp=tmp.sort_values('score', ascending=False)
tmp.head()

Rating               score
        count      mean          
Movie                            
2452   149866  4.434708  4.348818
3962   140979  4.415523  4.326062
4306   151292  4.325245  4.247792
3290    71614  4.404083  4.241464
2862   128446  4.311687  4.222690

In [69]:
tmp=tmp.merge(movie_titles , on='Movie')
tmp.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (2 levels on the left,1 on the right)



,"(Rating, count)","(Rating, mean)","(score, )",Year,Name
Movie,,,,,
2452,149866,4.434708,4.348818,2001.0,Lord of the Rings: The Fellowship of the Ring
3962,140979,4.415523,4.326062,2003.0,Finding Nemo (Widescreen)
4306,151292,4.325245,4.247792,1999.0,The Sixth Sense
3290,71614,4.404083,4.241464,1974.0,The Godfather
2862,128446,4.311687,4.222690,1991.0,The Silence of the Lambs
